In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, QTable, join, hstack, vstack, unique, Column, MaskedColumn, setdiff
from astropy.utils.masked import Masked
from astropy.coordinates import SkyCoord, Galactic, Angle
from astropy.time import Time
import astropy.units as u
from astropy.utils.diff import report_diff_values
from astropy.visualization import quantity_support, hist
quantity_support()
import glob as glob
import sys
import matplotlib.pyplot as plt
from plotnine import (
    ggplot,qplot, ggsave, aes,
    after_stat,
    stage,
    geom_point, geom_smooth, geom_qq, geom_bar, geom_text, geom_label,
    position_dodge2,
    facet_wrap,
    labs,
    scale_x_continuous, scale_x_log10, scale_y_log10,
    xlim, ylim,
    stat_qq
)
from scipy import optimize, stats
from sympy import symbols, exp, log, lambdify
import seaborn as sns
from custom_table import CustomQTable

In [ ]:
dt = QTable.read("./tables/final_table.ecsv")

set ggplot theme for matplotlib, seaborn


In [ ]:
sns.set_theme(style="darkgrid")

sns.set_palette("viridis")

::: abstract
The paper investigates the properties of galaxies in the Local
Cosmological Volume (LCV), using the Catalogue of Neighboring
[Galaxies\@karachentsevUPDATEDNEARBYGALAXY2013](mailto:Galaxies@karachentsevUPDATEDNEARBYGALAXY2013){.email}
and its updated version from the "Catalog & Atlas of the LV galaxies"
[database\@CatalogLVGalaxies](mailto:database@CatalogLVGalaxies){.email}.
The properties studied include the galaxy types, their various masses,
the star formation rates (SFRs) and the star formation timescale $\tau$,
gas depletion timescale $\tau_g$ and the star formation time $t_{sf}$.
The paper aims to understand the distribution and correlation of these
properties in the sample of galaxies in the LCV, and how they relate to
current astrophysical theories.
:::

# The Galaxies in the Local Cosmological Volume (LCV)

The Catalogue of Neigbouring Galaxies (Karachentsev, Igor D. and Makarov
et al.
[2013\@karachentsevUPDATEDNEARBYGALAXY2013](mailto:2013@karachentsevUPDATEDNEARBYGALAXY2013){.email})
and its updated version from the "Catalog & Atlas of the LV galaxies"
[databas\@CatalogLVGalaxies](mailto:databas@CatalogLVGalaxies){.email}
are used to extract the K-band luminosities, the types of the galaxies,
the mass within the Holmberg radius (M26), the Hydrogen masses of the
galaxies ($M_{HI}$) and the SFRs based on integrated H and
far-ultraviolet (FUV) measurments for galaxies within a distance of
$\approx 11$ Mpc.

## How are the galaxies chosen

According to @kraan-kortewegCatalogueGalaxiesIo1979 the Local
Cosmological Volume is defined as the galaxies inside the radius of 10
Mpc and having radial velocities with respect to centroid of the Local
Group $V_{lg} \le 500 \, km \cdot s^{-1}$. However, this assumed a
Hubble constant of $H_0 = 50\, km \cdot s^{-1}$.

1.  **Initial Selection Criteria**: Galaxies within a 10 Mpc radius were
    initially selected based on a radial velocity limit (VLG) of 500
    km/s, considering a Hubble parameter (H0) of 50 km/s/Mpc.

2.  **Updated Criteria**: To accommodate the revised H0 value of 73
    km/s/Mpc, the VLG limit needs to be raised to 730 km/s.

3.  **Local Velocity Field**: The presence of the Virgo cluster and the
    Local Void introduces additional velocity components, complicating
    distance estimation based solely on radial velocities.

4.  **Peculiar Motions**: Collective motions within large-scale
    structures can introduce peculiar velocities, complicating distance
    estimation.

5.  **Distance Measurement Methods**: Direct distance measurements using
    methods like the tip of the red giant branch (TRGB) provide accurate
    distances but are resource-intensive, requiring extensive
    observation time with instruments like the Hubble Space Telescope
    (HST).

6.  **Inclusion Criteria**: Galaxies are included based on either radial
    velocities or distance estimates, considering the limitations and
    uncertainties in both methods.

7.  **Extension to 11 Mpc**: Galaxies with distance estimates beyond 10
    Mpc may still be included due to uncertainties in distance
    measurements and the potential influence of coherent motions and
    large-scale structures.

8.  **Sample Composition**: The LV sample comprises
    src~python~$$:results value org$${len(dt)}
    {{{results(`1448`{.org})}}} galaxies, with considerations for
    galaxies near the boundaries of the selection criteria and the
    potential influence of measurement errors.


In [ ]:
hubble = (ggplot(dt.to_pandas())
          + aes( x = "Dis", y =  "VLG")
          + labs( x = f"Distance [{dt['Dis'].unit}]" , y = "Radial velocities $V_{LG} $"+f"[{dt['VLG'].unit:latex}]" )
          + xlim(0,11) + ylim(0,1200)
          + geom_smooth(color = "red")
          + geom_point()
          )
fname = "figure/hubble.png"
hubble.save(fname)
plt.show()
plt.close()

# Mapping the galaxies

Because matplotlib needs the coordinates in radians and between $-\pi$
and $\pi$ and, not 0 and $2\pi$, we have to convert coordinates.


In [ ]:
filename = "figure/mapping"

# Assuming dt is your data table containing coordinates, mass, and distance
c = dt["Coordinates"]
mass = dt["M26"].data  # Assuming mass is provided in some unit
distance = dt["Dis"].data  # Assuming distance is provided in some unit

# Extract Galactic Coordinates
galactic_coords = c.galactic

# Extract Equatorial Coordinates
equatorial_coords = c.transform_to('icrs')

# Define the size and color based on mass and distance
marker_size = np.sqrt(mass) * 0.0002  # Adjust scaling factor as needed
marker_color = distance   # Use distance directly for marker color

# Plot Galactic Coordinates
plt.figure(figsize=(8, 8))
plt.subplot(211, projection="aitoff")
plt.grid(True)
plt.scatter(galactic_coords.l.wrap_at(180 * u.deg).radian, galactic_coords.b.radian, s=marker_size, c=marker_color, cmap='viridis')
plt.colorbar(label='Distance [{}]'.format(dt['Dis'].unit))  # Add colorbar for distance
plt.title("Galactic Coordinates")

# Plot Equatorial Coordinates
plt.subplot(212, projection="mollweide")
plt.grid(True)
plt.scatter(equatorial_coords.ra.wrap_at(180 * u.deg).radian, equatorial_coords.dec.radian, s=marker_size, c=marker_color, cmap='viridis')
plt.colorbar(label='Distance [{}]'.format(dt['Dis'].unit))  # Add colorbar for distance
plt.title("Equatorial Coordinates")


plt.suptitle("Galaxies of the LCV with Mass and Distance Representation")

plt.tight_layout()  # Adjust spacing between subplots
plt.savefig(filename)
plt.close()

filename+".png"

## Types of galaxies

Using the dataset of src~python~$$:results value org :exports
results$${len(dt)} {{{results(`1448`{.org})}}} galaxies, we can study
the morphology of the galaxies in the LCV

### Morphology


In [ ]:
x="TType"
file="Types"
label="Morphology type code"
dttype = pd.DataFrame({"x": dt["{}".format(x)]}).dropna()
morphology = (
    ggplot(dttype, aes("factor(x)"))
    + geom_bar(color="black", fill="#1f77b4", show_legend=False)
    + geom_text(
        aes(label=after_stat("count")),
        stat="count",
        nudge_y=15,
        va="bottom",
        size = 9
    )
    + geom_text(
        aes(label=after_stat("prop*100"), group=1),
        stat="count",
        va="bottom",
        format_string="({:.1f}%)",
        size = 6
    )
    + labs(x = "{}".format(label))
)
fname = "figure/{}.png".format(file)
morphology.save(fname)

"[[./"+fname+"]]"

1.  Morphology of dwarf galaxies

2.  Dwarf galaxy surface brightness morphology

# Understanding the Data

The catalog consists of 8 tables

1.  Catalog of Nearby Galaxies
2.  Global Parameters of the Nearby Galaxies
3.  List of Apparent Magnitudes
4.  List of Heliocentric Velocities
5.  List of Inner Kinematics
6.  List of Distances
7.  List of the nearby galaxies with measured SFR
8.  List of Bibliographic References

We want several measurments from those lists so we will join them
according to the name of the galaxy.

This catalog consists of src~python~$$:results value org$${len(dt)+1}
{{{results(`1449`{.org})}}} galaxies

## Understanding the limit flags

Some of those values contain limit flags, which we will mask for our
present analysis. However, those values will be shown in the plots, and
afterwards will be compared with the theoretical values.


In [ ]:
for column in dt.columns:
    if column.startswith("l_") or column.startswith("f_"):
        if column.startswith("l_"):
            corresponding_column_name = column[2:]  # Remove the 'l_' prefix
        else:
            corresponding_column_name = column[2:]  # Remove the 'f_' prefix

        try:
            all_masks_in_corresponding = all(mask in dt[corresponding_column_name].mask for mask in dt[column].mask)
            if all_masks_in_corresponding:
                print(f"All masks in {column} are also masks in {corresponding_column_name}")
            else:
                print(f"Not all masks in {column} are masks in {corresponding_column_name}")
        except AttributeError:
            print(f"We have no mask for {column}")

# Standarized constants

We should use some standart consistent values for our analysis.

1.  According to @speagleHighlyConsistentFramework2014
    [and\@kroupaConstraintsStarFormation2020](mailto:and@kroupaConstraintsStarFormation2020){.email}
    the $t_{sf} = 12\, Gyr$ represents a strong and consistent
    constraint of galaxy evolution, across many studies. While other
    researchers adopt a t~sf~= 13.6
    [Gyr\@haslbauerCosmologicalStarFormation2023](mailto:Gyr@haslbauerCosmologicalStarFormation2023){.email},
    we use the 12 Gyr assumption following the framework of SP14
2.  $\zeta =$ accommodates mass-loss through stellar evolution.
    According to the IGIMF theory the galaxies of the the LCV are
    expected to have 1\< $\zeta$ \<1.3, so by adopting $\zeta =1.3$ we
    are working conservatively
3.  Main Sequence z = 5


In [ ]:
t_sf = 12 * u.Gyr
zeta = 1.3
z = 5

# Calculations for values that we need

## Total stellar masses, the total gas mass and total barionic of the galaxies

The $MHI$ can be converted to the total mass of the gas of the galaxy
using the equation $M_g=1.33\, MHI$

The K-band values are converted to the total Stellar Masses of each
galaxy according to the mass-to-light ratio of 0.6
($M_\odot/Lum$)@lelliSPARCMASSMODELS2016


In [ ]:
dt["StellarMass"] = 0.6 * dt["KLum"]* u.Msun/u.solLum
dt["StellarMass"].description = "K-band luminosity using a mass-to-light ratio of 0.6"
dt["StellarMass"].info()

The total barionic mass can be calcuated as the sum of the total gas
mass of the galaxy with the Stellar mass


In [ ]:
dt["BarMass"] = dt["M_g"] + dt["StellarMass"]

dt["BarMass"].info()

### Ratio of M~g~ and StellarMass


In [ ]:
dt["mass_ratio"] = dt["M_g"] / dt["StellarMass"]
dt["mass_ratio"].info(["attributes", "stats"])

Histogram of dt$$\"mass~ratio~\"$$


In [ ]:
#seaborn plot of mass_ratio
sns.histplot(dt["mass_ratio"].value)
plt.xscale("log")
plt.show()

## Color index

Here we calculate the color indexes \<FUV-B\>


In [ ]:
dt["color"] = dt["FUVmag"]-dt["Bmag"]

The lower the value, the bluer the stars, thus the younger the star
populations


In [ ]:
#hist
hist(dt["color"], bins = "freedman")
plt.xlabel("Color index")
plt.ylabel("Number of stars")
plt.title("Color index <FUV - B> distribution")
#save
plt.savefig("figure/color_index.png")
plt.close()
#print in org
"./figure/color_index.png"

## Fixing the SFRs

### SFR units


In [ ]:
SFR_unit = u.Msun/u.year

### log to linear

they are the power in logarithmic scale. SO lets fix them


In [ ]:
dt.rename_column("SFRFUV", "logSFRFUV")
dt["SFRFUV"] = 10**dt["logSFRFUV"].value * SFR_unit
dt["SFRFUV"].mask = dt["logSFRFUV"].mask
dt["SFRFUV"].description = dt["logSFRFUV"].description

dt["SFRFUV"].info()

In [ ]:
dt.rename_column("SFRHa", "logSFRHa")
dt["SFRHa"] = 10**dt["logSFRHa"].value * SFR_unit
dt["SFRHa"].mask = dt["logSFRHa"].mask
dt["SFRHa"].description = dt["logSFRHa"].description

In [ ]:
plt.scatter(dt["SFRHa"],dt["SFRFUV"])
plt.show()

In [ ]:
dt[["SFRFUV", 'SFRHa']].info("stats")

## SFR~0~

Now we have to calculate the total SFR from the equation:

$$
    SFR_o=\frac{SFR_{FUV}+SFR_{Ha}}{2}
$$

if we have both the SFR. If we only have one of them then:

$$
    SFR_{0}=SFR_{i},\ \text{if } SFR_{j}=0,\ i\neq j,\ i,j=SFR_{FUV},\, SFR_{Ha}
$$

create the average SFR~0~ from SFRHa SFRFUV with np.ma.average


In [ ]:
SFR_0 = np.mean([dt["SFRHa"], dt["SFRFUV"]], axis =0, where = [~dt["SFRHa"].mask, ~dt["SFRFUV"].mask])

dt["SFR_0"] = SFR_0 * SFR_unit

dt[["SFR_0","SFRFUV", 'SFRHa']].info("stats")

In [ ]:
print(dt["SFRHa"].info('stats'))

## Applying the cut SFR~0~ \>= 1e-3 solMass/yr

keep only the SFR~0~ data were \>1e-3


In [ ]:
dc = dt.copy()
dc = dc[dc["SFR_0"].value >= 1e-3]

print(dc["SFR_0"].info())

In [ ]:
dc[["SFR_0","SFRFUV", 'SFRHa']].info("stats")

Histogram of SFR~0~


In [ ]:
hist(dc["SFR_0"].value, bins = "scott")
plt.show()

## Theoretical Average SFR

To calculate the average Star Formation Rate $\overline{SFR}$ we can use
the equation

$$
    \overline{SFR}=\frac{\zeta M_*}{t_{sf}}
$$

where ζ is the mass-loss through stellar evolution and we assume that
$\zeta\approx 1.3$ (see explanation in the paper\`), M\* is the stellar
mass of each galaxy and we assume that is $t_{sf}=12.5\ Gyr$


In [ ]:
dc["av_SFR_theor"] = 1.3 * dc["StellarMass"] / t_sf.to(u.yr)
dc["av_SFR_theor"].info()

In [ ]:
plt.scatter(y = dc["av_SFR_theor"],x = dc["SFR_0"], c = dc["color"].value)
#colobar
plt.colorbar()

plt.xscale("log")
plt.yscale("log")
plt.show()

## Ratio av~SFR~/SFR~0~

Now we have to calculate the ratio $\frac{\overline{SFR}}{SFR_0}$


In [ ]:
dc["SFR_ratio"] = dc["av_SFR_theor"] / dc["SFR_0"]

#log10 of ratio
dc["logSFR_ratio"] = np.log10(dc["SFR_ratio"])

dc[["SFR_ratio", "logSFR_ratio"]].info(["attributes","stats"])

log10 of ratio


In [ ]:
plt.scatter(y = dc["SFR_ratio"],x = dc["SFR_0"], c = dc["color"].value)
#colobar
plt.colorbar()

plt.xscale("log")
plt.yscale("log")
plt.show()

Scatter color and ratio


In [ ]:
plt.scatter(y = dc["SFR_ratio"],x =dc["color"] )
#title and labels with units in latex
plt.yscale("log")
plt.title("(ratio) vs color")
plt.ylabel("(ratio)")
plt.xlabel("color")
#save in dir figure
plt.savefig("figure/ratio_vs_color.png")
plt.close()
#print file
print("figure/ratio_vs_color.png")

# The Delayed-τ model

"The delayed-τ model describes the SFH of a galaxy assuming that the
SFRs typically rise in the early phase of galaxy evolution and gradually
decline to the present time (e.g. Reddy et al. 2012; Carnall et al.
2019). In fact, Speagle et al. (2014) showed in their figures 9 and 10
that the SFH of galaxies following the main sequence of star-forming
galaxies can be accurately parametrized by the delayed-τ model of the
form" @haslbauerCosmologicalStarFormation2023


```{=latex}
\begin{equation}
        \label{eq:SFR} SFR_{0,del}=\frac{A_{del}xe^{-x}}{\tau},\text{ where } x=\frac{tsf}{\tau}
\end{equation}
```

`\noindent`{=latex} where

is the star formation time-scale, $tsf$ is the real time of star
formation in a given galaxy and $Adel$ a normalization constant.

The average SFR is


```{=latex}
\begin{equation}
        \label{eq:av_SFR-x} \overline{SFRdel}=\frac{Adel}{tsf}[1-(1+x)e^{-x}]
\end{equation}
```

and can also be defined by the present day stellar mass

$$\label{eq:av_SFR M*}
        \overline{SFR}=\frac{\zeta M_*}{tsf}$$

where accommodates for mass-loss through stella evolution and This is a
system of 2 equations and 3 variables

## Calculating A~del~

### Constant t~sf~

The observed ages of galactic discs are $tsf≈ 12$
[Gyr\@knoxSurveyCoolWhite1999a](mailto:Gyr@knoxSurveyCoolWhite1999a){.email},
so assuming an approximation of $tsf=12$ Gyr, the $\overline{SFR_{del}}$
can be calcuated, from the equation (`\ref{eq:av_SFR M*}`{=latex}).

After that the equation of ratio

$$\label{eq:ratio}                                        \frac{\overline{SFRdel}}{SFR0,del}=\frac{e^x-x-1}{x^2}$$

can be solved numerically for x and using the equations
(`\Ref{eq:SFR}`{=latex}) and (`\Ref{eq:av_SFR-x}`{=latex}) the $Adel$
and of each galaxy are found.


In [ ]:
dc["SFR_0", "SFR_ratio", "StellarMass"].info()

In [ ]:
ratio_array = np.array(dc["SFR_ratio"])
sfr_array = np.array(dc["SFR_0"])
mass_array = np.array(dc["StellarMass"])
tsf = t_sf/u.yr
x2 = np.empty(len(dc))

### Newton


In [ ]:
for i in range(len(dc)-1):
    ratio = ratio_array[i]
    mass = mass_array[i]
    sfr = sfr_array[i]
    def f(x):
        return (-sfr + zeta*mass*x**2/(np.exp(x)-1-x)/tsf )  # only one real root at x = 1

    def f_prime(x):
        return -zeta*mass*(x*(np.exp(x)*(x-2)+x+2)/(np.exp(x)-x-1)**2)/tsf

    sol = optimize.root_scalar(f, bracket=[0, 4], x0 = 3.4, fprime = f_prime, method="newton")
    x2[i] = sol.root

dc["x_n"] = x2
dc["A_n"] = dc["SFR_0"]*t_sf.to(u.yr)*np.exp(dc["x_n"])/(dc["x_n"]**2)

In [ ]:
print(dc["x_n", "A_n"].info("stats"))

In [ ]:
# Assuming you have already loaded your data into an Astropy table named 'dts'
# Replace 'x' with the name of the column you want to plot

# Create a histogram
hist(dc["x_n"], bins = "freedman")  # Adjust the number of bins as needed
plt.xlabel('X values')
plt.ylabel('Frequency')
plt.title('Histogram of Astropy Column "x"')
plt.show()

### fsolve


In [ ]:
from scipy.optimize import fsolve
# Example loop
x = np.ma.empty(len(dc))
A = np.ma.empty(len(dc))
for i in range(len(dc)-1):

    ratio = ratio_array[i]
    mass = mass_array[i]
    sfr = sfr_array[i]
    def sfrx(z):
        x = z[0]
        A = z[1]

        f = np.zeros(2)
        f[0] = ratio - (np.exp(x) - x - 1) / x**2
        f[1] = sfr - A * x * tsf * np.exp(-x) / x
        return f

    # Solve the equation
    z = fsolve(sfrx, [3, 10*9])
    x[i] = z[0]
    A[i] = z[1]

    ## mask if sfr ratio or mass is nan
    if np.isnan(ratio) or np.isnan(mass):
        x[i] = np.nan
        A[i] = np.nan

dc["x_f"] = MaskedColumn(x, name = "x")
dc["A_f"] = MaskedColumn(A, name = "A", unit = u.solMass)

In [ ]:
print(dc["x_f","A_f"].info(["attributes" ,"stats"]))

scatter of x2 and A

### Compare the methods


In [ ]:
plt.scatter(dc["x_f"], dc["A_f"])
#plt.scatter(dc["x_n"], dc["A_n"], alpha = 0.7)

plt.xlabel('x')
plt.yscale("log")
plt.title('scatter of x and A')
plt.show()

In [ ]:
hist(dc["x_f"], bins = "knuth")
hist(dc["x_n"], bins = "knuth", alpha = 0.6)
plt.xlabel('x')
plt.ylabel("Number of galaxies")
plt.title('Hist of x solved with fsolve and Newton')
#show the labels
plt.legend(["fsolve", "Newton"])

plt.savefig("figure/x-hist.png")
plt.close()

"figure/x-hist.png"

In [ ]:
print(dc["x","x2"].info(["attributes" ,"stats"]))

In [ ]:
plt.scatter(dc["x_f"], dc["x_n"])
plt.xlabel('$x_{fsolve}$')
plt.ylabel(r'$x_{Newton}$')
plt.title('scatter of $x_{Newton}$ and $x_{fsolve}$')
#savefig and print the file
plt.savefig("figure/x-scatter.png")
plt.close()

"figure/x-scatter.png"

Since they are both prety much the same, we can assume that the more
compact is better, ie fsolve.

### Hist of A


In [ ]:
hist(dc["A_f"], bins = "blocks")
hist(dc["A_n"], bins = "blocks")
plt.xscale("log")
plt.xlabel('$A_{del}$'+f'[{dc["A_n"].unit:latex}]')

plt.legend(["fsolve", "Newton"])

plt.ylabel("Number of galaxies")
plt.title('Histogram of $A_{del}$')
#save
plt.savefig("figure/A-hist.png")
plt.close()

"figure/A-hist.png"

## Calculating the t~sf~ and τ of the galaxies


In [ ]:
dc['tau'] = t_sf/dc["x_f"]

print(dc["tau"].info(["attributes", "stats"]))

In [ ]:
hist(dc["tau"].data, bins = "freedman")
plt.show()

In [ ]:
plt.scatter(dc["x_f"], dc["A_f"], c=np.log10(dc["SFR_0"]/dc["SFR_0"].unit), cmap = "viridis")
plt.title('Scatter plot of A vs x')
plt.xlabel(r'$\tau$ '+ f'[{dc["tau"].unit:latex}]')
plt.ylabel(r'$A$ '+ f'[{dc["A_f"].unit:latex}]')
plt.yscale("log")

plt.colorbar()
plt.show()

In [ ]:
print(dc["A_n", "x_n"].info())

In [ ]:
plt.scatter(dc["A_n"], dc["x_n"], c=np.log10(dc["StellarMass"]/dc["StellarMass"].unit), cmap = "viridis")
plt.title('Scatter plot of A vs x')
plt.xlabel(r'$\tau$ '+ f'[{dc["tau"].unit:latex}]')
plt.ylabel(r'$A$ '+ f'[{dc["A_f"].unit:latex}]')
plt.yscale("log")

plt.colorbar()
plt.show()

### IDEA Check to see if the almost inf points make any sense

## [TODO]{.todo .TODO} The gas depletion timescale τ~g~ {#the-gas-depletion-timescale-τg}

What is the gas depletion timescale?

## [TODO]{.todo .TODO} The theoretical SFR vs the observed {#the-theoretical-sfr-vs-the-observed}

# PROJ The relations of the Masses

Since the aim of the paper is to find the SFR lets first understand and
calculate the masses of the galaxies and see if we can find any relation
with the SFR.


In [ ]:
df = dc.to_pandas()
# Assuming df is your DataFrame
# Step 1: Identify non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['float', 'int']).columns

# Step 2: Drop non-numeric columns or handle them appropriately
df_numeric = df.drop(columns=non_numeric_columns)
# Step 3: Replace NaN values with zeros or other appropriate values

df_numeric = df_numeric.loc[:, ~df_numeric.columns.str.startswith('e_')]

df_numeric = df_numeric.loc[:, ~df_numeric.columns.str.startswith('Name')]
df_numeric = df_numeric.loc[:, ~df_numeric.columns.str.startswith('Coord')]
df_numeric = df_numeric.loc[:, ~df_numeric.columns.str.startswith('log')]
# Step 4: Calculate the correlation matrix
correlation_matrix = df_numeric.corr()

plt.close("all")

In [ ]:
# Plot heatmap using seaborn
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# Set x-axis tick labels
plt.xticks(ticks=np.arange(0.5, len(correlation_matrix.columns)), labels=correlation_matrix.columns, rotation=90)

# Set y-axis tick labels
plt.yticks(ticks=np.arange(0.5, len(correlation_matrix.index)), labels=correlation_matrix.index, rotation=0)
plt.title('Correlation Heatmap')
plt.savefig("figure/heatmap")
plt.show()
plt.close("all")

In [ ]:
clustermap = sns.clustermap(correlation_matrix, cmap='coolwarm', fmt=".2f", linewidths=0.5,
                             row_cluster=False, col_cluster=False)

ax = clustermap.ax_heatmap

# Set x-axis tick labels
ax.set_xticks(np.arange(0.5, len(correlation_matrix.columns)))
ax.set_xticklabels(correlation_matrix.columns, rotation=90)

# Set y-axis tick labels
ax.set_yticks(np.arange(0.5, len(correlation_matrix.index)))
ax.set_yticklabels(correlation_matrix.index, rotation=0)


plt.savefig("figure/clustermap")
plt.show()
plt.close("all")

Pairplot with StellarMass, MHI, SFR~0~ and av~SFR~, M26


In [ ]:
#PairGrid with StellarMass, MHI, SFR_0 and av_SFR_theor, M26
#log scale axes

sns.pairplot(df, vars=["StellarMass", "MHI", "SFR_0", "av_SFR_theor", "M26"], kind="reg", diag_kind="kde")
plt.savefig("figure/pairplot")
plt.show()
plt.close("all")
print(df[["StellarMass", "MHI", "SFR_0", "av_SFR_theor", "M26", "M_g", "tau", "A"]].corr())

# [TODO]{.todo .TODO} The relations of the Data {#the-relations-of-the-data}

## [TODO]{.todo .TODO} Luminosity and Masses {#luminosity-and-masses}

## [TODO]{.todo .TODO} Variations in Star Formation Rates across the different masses {#variations-in-star-formation-rates-across-the-different-masses}

# [TODO]{.todo .TODO} Filling the Catalogue {#filling-the-catalogue}